In [ ]:
# Conteggio scartati
max_size = 5200
print(f'dev_X scartati: {len([el.size for el in dev_X if el.size > max_size])/len(dev_X)*100:.2f} %')
print(f'eval_X scartati: {len([el.size for el in eval_X if el.size > max_size])/len(eval_X)*100:.2f} %')

In [ ]:
# Uniformation of signals: zero padding (at the end)

# Developement dataset
X_d = np.array([])
y_d = np.array([])
for sig,sig_class in zip(dev_X, dev_y):
    #Non considero gli outliers
    if sig.size > max_size:
        continue
        
    v = np.zeros(max_size)
    v[:sig.size] = sig
    X_d = np.vstack((X_d, v)) if X_d.size > 0 else v
    y_d = np.append(y_d, sig_class)

# Evaluation dataset
X_e = np.array([])
for sig in eval_X:
    #Non considero gli outliers
    if sig.size > max_size:
        continue
        
    v = np.zeros(max_size)
    v[:sig.size] = sig
    X_e = np.vstack((X_e, v)) if X_e.size > 0 else v

In [ ]:
# Amplitude rescaled to 1
for i in range(X_d.shape[0]):
    X_d[i,:] = X_d[i,:] / np.max(X_d[i,:])

for i in range(X_e.shape[0]):
    X_e[i,:] = X_e[i,:] / np.max(X_e[i,:])

In [ ]:
from scipy import fftpack

def freq_cleaner(sig, sampling_rate, cutoff=.2, power_plot=False, clean_power_plot=False):
    """
    Return: filtered_signal, bilateral_frequencies, bilateral_filtered_amplitude, bilateral_amplitude
    Note: power = np.abs(bilateral_amplitude)
          positive_frequencies = bilateral_frequencies[:bilateral_frequencies.size // 2]
    """

    # Signal
    time_step = 1/sampling_rate

    # The FFT of the signal
    sig_fft = fftpack.fft(sig)

    # And the power (sig_fft is of complex dtype)
    power = np.abs(sig_fft)

    # The corresponding frequencies
    sample_freq = fftpack.fftfreq(sig.size, d=time_step)
    
    if power_plot is True:
        # Plot the FFT power
        plt.figure(figsize=(15, 5))
        plt.plot(sample_freq, power) #plt.plot(sample_freq[0:1663], power[0:1663])
        plt.xlabel('Frequency [Hz]');
        plt.ylabel('power');
    
    # Removing high frequencies
    high_freq_fft = sig_fft.copy()
    high_freq_fft[(power / np.max(power)) < cutoff] = 0
    
    if clean_power_plot is True:
        # Plot the FFT power
        plt.figure(figsize=(15, 5))
        plt.plot(sample_freq, np.abs(high_freq_fft))
        plt.xlabel('Frequency [Hz]');
        plt.ylabel('power');
    
    filtered_sig = fftpack.ifft(high_freq_fft)

    #plt.figure(figsize=(15, 5))
    #plt.plot(sig, label='Original signal')
    #plt.plot(filtered_sig, linewidth=3, label='Filtered signal')
    #plt.xlabel('Time [s]')
    #plt.ylabel('Amplitude')
    #plt.legend(loc='best')
    
    return filtered_sig, sample_freq, high_freq_fft, sig_fft

In [ ]:
#Descr: lot per 10 segnali di una certa classe il dominio del tempo e dominio della freq. per ciascun segnale

import seaborn as sns

class_lab = '6'
signals = X_d[y_d == class_lab]

n_sig = 10
fig, ax = plt.subplots(n_sig, 2, figsize=(14, 30))

mean_pot_sig = None

for i, sig in enumerate(signals[:n_sig]):
    filt_sig, freq, filt_freq_amp, freq_amp = freq_cleaner(sig, sampling_rate)
    
    # Potenza
    filt_pot = np.abs(filt_freq_amp)
    pot = np.abs(freq_amp)
    
    #mean_time_sig = mean_time_sig + filt_sig / n_sig if mean_time_sig is not None else filt_sig / n_sig
    mean_pot_sig = mean_pot_sig + filt_pot / n_sig if mean_pot_sig is not None else filt_pot / n_sig
    
    #ax[i][0].plot(filt_sig)
    #ax[i][0].set_title('Segnale filtrato')
    ax[i][0].plot(sig)
    ax[i][0].set_title('Segnale originale')
    #ax[i][0].plot(freq, pot)
    #ax[i][0].set_title('Potenza originale')
    ax[i][1].plot(freq, filt_pot)
    ax[i][1].set_title('Potenza filtrata')
    
plt.tight_layout()
plt.show()

sns.heatmap([mean_pot_sig[:mean_pot_sig.size//2]]);

In [37]:
import numpy as np

def componenti_connesse(connessioni):
    """
    Restituisce le componenti connesse a partire da una lista di connessioni
    connessioni:lista di liste
    """
    elem = []
    for c in connessioni:
        elem.append(c[0])
        elem.append(c[1])
        
    # Vettore di elementi: possono essere qualsiasi oggetto
    elem = list(set(elem))
    
    # Vettore delle connessioni
    v = np.array([i for i in range(len(elem))])
    
    for c in connessioni:
        # Hashing
        i = elem.index(c[0])
        j = elem.index(c[1])
        
        # Verifica connessione
        if v[i] != v[j]:
            
            # Creazione connessione
            b = v[j]
            a = v[i]
            for k in range(v.size):
                if v[k] == a:
                    v[k] = b
   
    # Estrazione componenti connesse
    cc = list(set(v))
    elem = np.array(elem)
    cc_el = []
    for c in cc:
        cc_el.append(elem[v == c].tolist())
    
    return cc_el

In [38]:
l = [['a', 'b'],
     ['d', 'e'],
     ['a', 'c'],
     ['a','a']
    ]


componenti_connesse(l)

[['a', 'c', 'b'], ['d', 'e']]

In [41]:
import pandas as pd
x = pd.DataFrame([{'a':1, 'b':2},
                  {'a':4, 'b':3}
                 ])
x

,a,b
0,1,2
1,4,3


In [43]:
x = np.array([0,])

""
0
1
